Cyna Shirazinejad, 1/26/22

# Notebook 11b: merge AP2 with ARPC3, 'zero' padding

outline:
* find ARPC3+/- events
* measure the effect of CCP motility with ARPC3 recruitment

##### import all necessary Python modules

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import pickle
from IPython.display import Image
from scipy import signal
import scipy.stats as stats
import seaborn as sns

unique_user_saved_outputs = str(np.load('unique_user_saved_outputs_simplified.npy'))
analysis_metadata = np.load(unique_user_saved_outputs+'/dataframes/analysis_metadata.npy', allow_pickle=True)
sys.path.append(analysis_metadata.item().get('path_notebook')+'/cmeAnalysisPostProcessingSimplified') # add custom Python scripts to the local path
import separate_tracking_merge_tools
import merge_tools
import generate_index_dictionary
import return_track_attributes
index_dictionary = generate_index_dictionary.return_index_dictionary()

In [2]:
analysis_metadata

array({'path_outputs': '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_simplified_new', 'path_tracks_ad_wildtype': '/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/tracking_data/AD_cellline_analysis_formatted', 'path_notebook': '/Users/cynashirazinejad/Documents/GitHub/Jin_Shirazinejad_et_al_branched_actin_manuscript/analysis/simplified_workflow', 'feature_units': ['seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'pixels', 'pixels', 'pixels', 'pixels', 'pixels', 'seconds', 'seconds', 'seconds', 'seconds', 'seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'unitless', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence**2', 'a.u. fluorescence**2', 'unitless', 'unitless', 'unitless', 'unitless', 'counts', 'counts', 'unitless', 'unitless', 'unitless'], 'possible_track_features': ['lifetime', 'max_int_ch0', 'max_int_ch1', 'dist_traveled_ch0', 'dist_traveled_ch1', 'ma

# load dataframe from notebook 3 containing normal-pdf scaled features: PC's and GMM predicted clusters, and dataframe with cmeAnalysis labels

In [3]:
# df_pcs_normal_scaled_with_gmm_cluster = pd.read_csv(unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_pcs_gmm_clusters.zip')
# df_merged_features = pd.read_csv(unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_merged_features.zip')
# index_dnm2positive = np.load(unique_user_saved_outputs+'/dataframes/cluster_dnm2_positive.npy')
# number_of_track_splits = np.load(unique_user_saved_outputs+'/dataframes/number_of_track_splits.npy')
# number_of_clusters = np.load(unique_user_saved_outputs+"/dataframes/number_of_clusters.npy")
# ccp_predictions = np.load(unique_user_saved_outputs+'/dataframes/merged_ccp_predictions.npy')

In [4]:
# best_fit_peak_params = np.load(unique_user_saved_outputs+'/dataframes/parameters_best_fit_peak_finding.npy')
# sos = signal.butter(4, 0.2, 'lp', fs=1, output='sos')

In [5]:
# df_merged_features

# load all valid arpc3 tracks

In [6]:
unique_user_saved_outputs

'/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_simplified_new'

In [48]:
valid_tracks_arpc3 = [np.load(unique_user_saved_outputs+'/dataframes/arpc3_hypotonic_merged_before_separate_arpc3_tracking_0.npy', allow_pickle=True)]
print(len(np.load(unique_user_saved_outputs+'/dataframes/arpc3_hypotonic_merged_before_separate_arpc3_tracking_0.npy', allow_pickle=True)))
num_arpc3_tracks = 10
for i in range(1, num_arpc3_tracks):
    print(i)
    valid_tracks_arpc3 += [np.load(unique_user_saved_outputs+'/dataframes/arpc3_hypotonic_merged_before_separate_arpc3_tracking_'+str(i)+'.npy', allow_pickle=True)]
    print(len(np.load(unique_user_saved_outputs+'/dataframes/arpc3_hypotonic_merged_before_separate_arpc3_tracking_'+str(i)+'.npy', allow_pickle=True)))
    
    

13557
1
12428
2
14836
3
13362
4
14716
5
15038
6
16510
7
14837
8
13944
9
13399


In [51]:
len(valid_tracks_arpc3[0])

13557

In [52]:
merged_valid_tracks_arpc3 = merge_tools.merge_experiments(valid_tracks_arpc3,[list(range(len(track_set))) for track_set in valid_tracks_arpc3])

# load all valid tracks ap2/dnm2 tracks from arpc3 line

In [53]:
merged_all_valid_tracks = np.load(unique_user_saved_outputs+'/dataframes/merged_all_valid_ada_shock_before_both_dates_tracks_0.npy', allow_pickle=True)

for i in range(1,10):
    print(i)
    merged_all_valid_tracks = np.concatenate((merged_all_valid_tracks,
                                              np.load(unique_user_saved_outputs+'/dataframes/merged_all_valid_ada_shock_before_both_dates_tracks_'+str(i)+'.npy', allow_pickle=True)))
    

1
2
3
4
5
6
7
8
9


In [11]:
merged_all_valid_tracks.shape

(74292,)

# sort DNM2+ events

In [12]:
number_of_clusters = analysis_metadata.item().get('number_of_clusters')

num_gmm_clusters = number_of_clusters # optimal number of clusters of PCA data


gmm_classes = []

df = pd.read_csv(unique_user_saved_outputs+'/dataframes/df_ada_shock_before_both_dates_merged_features.zip')

for i in range(num_gmm_clusters):

    gmm_classes.append(df[df['gmm_predictions']==i].index.values)    

In [13]:
df['experiment_number']

0        22.0
1        22.0
2        22.0
3        22.0
4        22.0
         ... 
74287    70.0
74288    70.0
74289    70.0
74290    70.0
74291    70.0
Name: experiment_number, Length: 74292, dtype: float64

In [14]:
index_dnm2positive = analysis_metadata.item().get('index_DNM2positive')

In [15]:
len(gmm_classes[index_dnm2positive])

5593

In [16]:
tracks_authentic_ccps_gmm = merged_all_valid_tracks[gmm_classes[index_dnm2positive]] #dnm2 positive events, mixed CCPS and hot-spots

In [17]:
len(tracks_authentic_ccps_gmm)

5593

# separate single-CCP events by experiment

In [18]:
len(np.where(df['ccp_status']==1)[0])

1978

In [30]:
experiment_numbers = list(range(22, 31, 2)) + list(range(62, 71, 2))

In [31]:
experiment_numbers

[22, 24, 26, 28, 30, 62, 64, 66, 68, 70]

In [36]:
merged_all_valid_tracks_per_experiment = [] # each entry is a list for each experiment of tracks corresponding to predicted ccps 

ccp_events_per_experiment = []

for experiment_number in experiment_numbers: # only look at 3-color movies
    print(experiment_number)
    tracks_exp_idx = df[df['experiment_number']==experiment_number].index.values
    tracks_exp = merged_all_valid_tracks[tracks_exp_idx]
    
    ccp_preds_exp = df['ccp_status'].values[tracks_exp_idx]
    idx_ccp_exp = np.where(ccp_preds_exp==1)[0]
    
    ccp_events_per_experiment.append(tracks_exp[idx_ccp_exp])

22
24
26
28
30
62
64
66
68
70


<!-- # measure ap2 initiation to dnm2 peak lifetime, comparing 2 cell lines -->

# merge all CCPs

In [37]:
merged_ccps = merge_tools.merge_experiments(ccp_events_per_experiment, [list(range(len(track_set))) for track_set in ccp_events_per_experiment])

In [38]:
len(merged_ccps)

1978

# reload (or load) trees 

In [54]:
arpc3_trees = []

string_index_matrix = 'arpc3_hypotonic_merged_before_separate_arpc3_tracking_kdtree_index_matrix_'

string_tree = 'arpc3_hypotonic_merged_before_separate_arpc3_tracking_kdtree_'

for i in range(0,10):
    print(i)
    with open(unique_user_saved_outputs+'/dataframes/'+string_tree+str(i), 'rb') as f:
        tree = pickle.load(f)
        
    index_matrix = np.load(unique_user_saved_outputs+'/dataframes/'+string_index_matrix+str(i)+'.npy', allow_pickle=True)

    arpc3_trees.append([tree, index_matrix])

0
1
2
3
4
5
6
7
8
9


In [55]:
sos = signal.butter(4, 0.2, 'lp', fs=1, output='sos')

In [56]:
dist_fit = analysis_metadata.item().get('distance_best_fit')
height_fit = analysis_metadata.item().get('height_best_fit')
width_fit = analysis_metadata.item().get('width_best_fit')

best_fit_peak_params = [dist_fit,
                        height_fit,
                        width_fit]

In [58]:
distance_query = 2 # pixel search radius for associating ap2 and arpc3 from separate tracking

# arpc3 positive events, padded in non-overlapping frames
ch0_vectors_arpc3_positive = [] # amplitudes #
ch1_vectors_arpc3_positive = [] #
ch2_vectors_arpc3_positive = [] #
channel_0_1_separations_arpc3_positive = [] # distance in microns between ap2 and dnm2
channel_0_2_separations_arpc3_positive = [] # distance in microns between ap2 and arpc3
channel_1_2_separations_arpc3_positive = []
final_separations_ap2arpc3_arpc3_positive = [] # ap2 and arpc3 final separation in last overlapping frame of ap2 and arpc3
initial_separations_ap2arpc3_arpc3_positive = [] # ap2 and arpc3 initial separation in first overlapping frame of ap2 and arpc3
ap2_arpc3_separation_at_dnm2_peak_arpc3_positive = [] # ap2 and arpc3 separation at peak of dnm2 signal
ap2_arpc3_separation_at_arpc3_peak_arpc3_positive = []
ap2_arpc3_separation_at_ap2_peak_arpc3_positive = []
ap2_lifetime_arpc3_positive = [] # ap2 lifetime
arpc3_lifetime_arpc3_positive = []
max_separation_ap2_arpc3_arpc3_positive = [] # maximum separation between ap2 and arpc3
max_separation_ap2_dnm2_arpc3_positive = []
max_separation_dnm2_arpc3_arpc3_positive = []
num_frames_associated_arpc3_positive = [] # number of frames shared between ap2 and arpc3 in movie that are below the query radius in KDTree search
time_ap2_appearance_to_dnm2_peak_arpc3_positive = []
maximum_ap2_intensity_arpc3_positive = []
maximum_dnm2_intensity_arpc3_positive = []
maximum_arpc3_intensity_arpc3_positive = []
pval_arpc3_ccps_arpc3_positive = []
time_arpc3_appearance_to_dnm2_peak_arpc3_positive = []
fraction_arpc3_positive = []
ccps_arpc3_positive = []
average_ap2_movement_before_dnm2_peak_arpc3_positive = []
average_ap2_movement_after_dnm2_peak_arpc3_positive = []

ap2_arpc3_separation_average_arpc3_positive = []
experiment_number_arpc3_positive = []
dnm2_arpc3_peak_time_difference_arpc3_positive = []


# arpc3 negative events
ch0_vectors_arpc3_negative = []
ch1_vectors_arpc3_negative = []
channel_0_1_separations_arpc3_negative = []
ccps_arpc3_negative = []
time_ap2_appearance_to_dnm2_peak_arpc3_negative = []
ap2_lifetime_arpc3_negative = []
experiment_number_arpc3_negative = []
average_ap2_movement_before_dnm2_peak_arpc3_negative = []
average_ap2_movement_after_dnm2_peak_arpc3_negative = []


num_frames_associated = []


significant_arpc3_at_dnm2_peak_arpc3_positive = []

num_no_neighbors = 0

for exp_idx , experiment_number in enumerate(experiment_numbers):  # iterate over all experiments

    
    num_with_mode = 0 # count number of events with and without arpc3
    num_without_mode = 0    
    num_no_neighbors=0
    print('current experiment number: ' + str(experiment_number))
    ap2dmn2_tracks_in_experiment = ccp_events_per_experiment[exp_idx] # filtered tracks
    print('number of ap2 tracks: ' + str(len(ap2dmn2_tracks_in_experiment)))
    arpc3_experiment=valid_tracks_arpc3[exp_idx] # cat 1 and 2 arpc3 events in experiment
    print('number of arpc3 tracks: ' + str(len(arpc3_experiment)))

    
    # select a tree for each frame containing all events in each frame
    kd_tree_arpc3_experiment, vals_tree = arpc3_trees[exp_idx]
    
    print('iterating through tracks in experiment')
    
    for track_num in range(len(ap2dmn2_tracks_in_experiment)): # iterate through all ap2 tracks in experiment

        
        frames_in_track_first_channel = list(return_track_attributes.return_frames_in_track_no_buffer(ap2dmn2_tracks_in_experiment, track_num)-1) # frames of ap2 and dnm2
        ch0_x = return_track_attributes.return_puncta_x_position_no_buffer_one_channel(ap2dmn2_tracks_in_experiment, track_num, 0) # positions of ap2 and dnm2
        ch0_y = return_track_attributes.return_puncta_y_position_no_buffer_one_channel(ap2dmn2_tracks_in_experiment, track_num, 0)
        ch1_x = return_track_attributes.return_puncta_x_position_no_buffer_one_channel(ap2dmn2_tracks_in_experiment, track_num, 1)
        ch1_y = return_track_attributes.return_puncta_y_position_no_buffer_one_channel(ap2dmn2_tracks_in_experiment, track_num, 1)
        ap2_int = return_track_attributes.return_track_amplitude_no_buffer_channel(ap2dmn2_tracks_in_experiment, track_num, 0) # ap2 and dnm2 intensities
        dnm2_int = return_track_attributes.return_track_amplitude_no_buffer_channel(ap2dmn2_tracks_in_experiment, track_num, 1)
        
        filtered_dnm2_signal = list(list(signal.sosfilt(sos, dnm2_int)) + [0, 0, 0, 0, 0])
    
        dnm2_peak = signal.find_peaks(filtered_dnm2_signal, 
                                      distance=best_fit_peak_params[0], 
                                      height=best_fit_peak_params[1],
                                      width=best_fit_peak_params[2])[0][0]
        
        frame_dnm2_peak = frames_in_track_first_channel[dnm2_peak] # the movie frame (indexed starting from 0) that dnm2 peaks     
        candidate_arpc3_neighbors = [] # indices of arpc3 events that take place near ap2
        no_neighbors=True # no neighbors found yet
        
        # search for candidate arpc3 neighbors in each ap2 track's frame
        for frame in frames_in_track_first_channel:
            
            frame_ch0 = frames_in_track_first_channel.index(frame)
            # frame's tree
            current_tree = kd_tree_arpc3_experiment[frame]
            
            # ap2 fitted position at current frame
            current_ap2_position = np.array([ch0_x[frame_ch0], ch0_y[frame_ch0]]).reshape(1, -1)
            
            # indices of arpc3 events within the query radius
            ind = current_tree.query_radius(current_ap2_position,
                                            r=distance_query)

            if len(ind[0])>0:
                no_neighbors = False # if there is a neighbor
                
            # check if each arpc3 neighbor originated before or after the ap2 event
            for candidate in ind[0]:

                candidate_index = int(vals_tree[frame][candidate][0]) # get the index of the arpc3 event
                
                frames_in_track_candidate_neighbor = list(return_track_attributes.return_frames_in_track_no_buffer(arpc3_experiment, candidate_index)-1)
                
                # consider the arpc3 event if it originated after the ap2 event and before the dnm2 peak
                if frames_in_track_candidate_neighbor[0]>frames_in_track_first_channel[0] and frames_in_track_candidate_neighbor[0]<frame_dnm2_peak:

                    candidate_arpc3_neighbors.append(candidate_index)
        
            
        if no_neighbors==True: # if there are absolutely no neighbors present, the CCP is arpc3 negative
            
            num_no_neighbors+=1
            
        # get mode(s) of candidates
        mode=stats.mode(candidate_arpc3_neighbors)
        
        # if there is a unique candidate:
        if mode[0].size == 1:

            num_frames_associated.append(mode[1][0])
            
            num_with_mode+=1 # count this track as arpc3 positive

            pval_arpc3_detection = []
            
        
            associated_track_second_channel = mode[0][0] # index of arpc3 track associated with current ap2 track
            
            frames_in_track_second_channel = list(return_track_attributes.return_frames_in_track_no_buffer(arpc3_experiment, associated_track_second_channel)-1) # frames of arpc3 track
            
            frames_intersection = list(set(frames_in_track_first_channel) & set(frames_in_track_second_channel)) # find all frames ap2 and arpc3 overlap during movie
            frames_intersection.sort() # sort the frames in ascending order

            # amplitudes of all channels
            ch0_amplitudes = ap2_int # ap2, dnm2, arpc3 intensities
            ch1_amplitudes = dnm2_int
            ch2_amplitudes = return_track_attributes.return_track_amplitude_no_buffer_channel(arpc3_experiment, associated_track_second_channel, 0)

            # position of arpc3
            ch2_x = return_track_attributes.return_puncta_x_position_no_buffer_one_channel(arpc3_experiment, associated_track_second_channel, 0)
            ch2_y = return_track_attributes.return_puncta_y_position_no_buffer_one_channel(arpc3_experiment, associated_track_second_channel, 0)

            # correct intensities and positions for missing frames where ap2 and arpc3 are non-overlapping
            new_ch0_vector = []
            new_ch1_vector = []
            new_ch2_vector = []
            new_ch0_x = []
            new_ch0_y = []
            new_ch1_x = []
            new_ch1_y = []
            new_ch2_x = []
            new_ch2_y = []
            
            # first/last frame either ap2 or arpc3 shows up
            min_frame = np.nanmin([np.nanmin(frames_in_track_first_channel),np.nanmin(frames_in_track_second_channel)]) #nan to account for gaps
            max_frame = np.nanmax([np.nanmax(frames_in_track_first_channel),np.nanmax(frames_in_track_second_channel)])

            # all frames shared between ap2 and arpc3
            frames_all = [i for i in range(int(min_frame),int(max_frame)+1)]

            channel_0_2_separation_track = [] # ap2 and arpc3 separation
            channel_0_2_angle_track = [] # ap2 and arpc3 angle 
            channel_0_1_separation_track = [] # ap2 and dnm2 separation
            channel_1_2_separation_track = []

            for frame in frames_all: # iterate over all frames

                # if frames of ap2 and arpc3 are non-overlapping, do not measure their separation
                if (frame in frames_in_track_first_channel and frame not in frames_in_track_second_channel) or (frame in frames_in_track_second_channel and frame not in frames_in_track_first_channel):
                    channel_0_2_separation_track.append(0)
                    channel_0_2_angle_track.append(0)
                    channel_1_2_separation_track.append(0)
                # if only ap2 is present
                if frame in frames_in_track_first_channel and frame not in frames_in_track_second_channel:
                    
                    channel_0_index = frames_in_track_first_channel.index(frame) # get relative index of ap2 in this frame 

                    new_ch0_vector.append(ch0_amplitudes[channel_0_index]) # add ap2 and dnm2 amplitudes
                    new_ch1_vector.append(ch1_amplitudes[channel_0_index]) 
                    new_ch0_x.append(ch0_x[channel_0_index]) # ap2 and dnm2 positions
                    new_ch0_y.append(ch0_y[channel_0_index])
                    new_ch1_x.append(ch1_x[channel_0_index])
                    new_ch1_y.append(ch1_y[channel_0_index])

                    new_ch2_vector.append(0) # no arpc3
                    new_ch2_x.append(0) # arpc3 position off grid
                    new_ch2_y.append(0)
                    pval_arpc3_detection.append(np.NaN)
                    
                    channel_0_1_separation_track.append(np.sqrt((0.108*(ch0_x[channel_0_index]-ch1_x[channel_0_index]))**2 + (0.108*(ch0_y[channel_0_index]-ch1_y[channel_0_index]))**2)) # ap2-dnm2 separation

                # if only arpc3 is present
                elif frame in frames_in_track_second_channel and frame not in frames_in_track_first_channel: 

                    channel_2_index = frames_in_track_second_channel.index(frame) # get relative index of arpc3 in this frame 

                    new_ch2_vector.append(ch2_amplitudes[channel_2_index]) # add arpc3 amplitudes and positions
                    new_ch2_x.append(ch2_x[channel_2_index]) 
                    new_ch2_y.append(ch2_y[channel_2_index])
                    
                    new_ch0_vector.append(0) # no ap2 or dnm2
                    new_ch1_vector.append(0)    
                    new_ch0_x.append(0) # ap2 and dnm2 position off grid
                    new_ch0_y.append(0)
                    new_ch1_x.append(0)
                    new_ch1_y.append(0)
           
                    channel_0_1_separation_track.append(0) # no separation between ap2 and dnm2 
                    
                    pval_arpc3_detection.append(return_track_attributes.return_pvals_detection_no_buffer(arpc3_experiment, associated_track_second_channel, 0)[channel_2_index])
                
                # if both ap2 and arpc3 in frame
                elif frame in frames_in_track_first_channel and frame in frames_in_track_second_channel:  

                    channel_0_index = frames_in_track_first_channel.index(frame) # get relative indices
                    channel_2_index = frames_in_track_second_channel.index(frame)

                    new_ch0_vector.append(ch0_amplitudes[channel_0_index]) # add amplitudes and positionsfor all channels
                    new_ch1_vector.append(ch1_amplitudes[channel_0_index])
                    new_ch2_vector.append(ch2_amplitudes[channel_2_index])
                    new_ch0_x.append(ch0_x[channel_0_index]) # ap2 and dnm2 positions
                    new_ch0_y.append(ch0_y[channel_0_index])
                    new_ch1_x.append(ch1_x[channel_0_index])
                    new_ch1_y.append(ch1_y[channel_0_index])
                    new_ch2_x.append(ch2_x[channel_2_index]) 
                    new_ch2_y.append(ch2_y[channel_2_index])

                    channel_0_1_separation_track.append(np.sqrt((0.108*(ch0_x[channel_0_index]-ch1_x[channel_0_index]))**2 + (0.108*(ch0_y[channel_0_index]-ch1_y[channel_0_index]))**2)) # ap2-dnm2 separation
                    
                    # ap2 and arpc3 separation
                    channel_0_2_separation_track.append(np.sqrt((0.108*(ch0_x[channel_0_index]-ch2_x[channel_2_index]))**2 + (0.108*(ch0_y[channel_0_index]-ch2_y[channel_2_index]))**2))
#                     channel_0_2_angle_track.append(angle_between((0,0),((ch0_x[channel_0_index]-ch2_x[channel_2_index]),(ch0_y[channel_0_index]-ch2_y[channel_2_index]))))              
                    channel_1_2_separation_track.append(np.sqrt((0.108*(ch1_x[channel_0_index]-ch2_x[channel_2_index]))**2 + (0.108*(ch1_y[channel_0_index]-ch2_y[channel_2_index]))**2))

                    pval_arpc3_detection.append(return_track_attributes.return_pvals_detection_no_buffer(arpc3_experiment,associated_track_second_channel,0)[channel_2_index])
 
            filtered_dnm2_signal = list(list(signal.sosfilt(sos, dnm2_int)) + [0, 0, 0, 0, 0])
    
            dnm2_peak = signal.find_peaks(filtered_dnm2_signal, 
                                          distance=best_fit_peak_params[0], 
                                          height=best_fit_peak_params[1],
                                          width=best_fit_peak_params[2])[0][0]
        
            if pval_arpc3_detection[dnm2_peak]<0.01 and pval_arpc3_detection[dnm2_peak]!=np.NaN:
            
                significant_arpc3_at_dnm2_peak_arpc3_positive.append(1)
            
            else:
                
                significant_arpc3_at_dnm2_peak_arpc3_positive.append(0)
    
            ccps_arpc3_positive.append(ap2dmn2_tracks_in_experiment[track_num])
            ap2_lifetime_arpc3_positive.append(len(frames_in_track_first_channel))
            arpc3_lifetime_arpc3_positive.append(len(frames_in_track_second_channel))
            # measure ap2 and arpc3 separation at dnm2 peak

            ap2_arpc3_separation_at_dnm2_peak_arpc3_positive.append(channel_0_2_separation_track[dnm2_peak])
            ap2_arpc3_separation_at_arpc3_peak_arpc3_positive.append(channel_0_2_separation_track[np.nanargmax(new_ch2_vector)])
            ap2_arpc3_separation_at_ap2_peak_arpc3_positive.append(channel_0_2_separation_track[np.nanargmax(new_ch0_vector)])
            ap2_arpc3_separation_average_arpc3_positive.append(np.nanmean(channel_0_2_separation_track))
            # store measurements
            ch0_vectors_arpc3_positive.append(new_ch0_vector)
            ch1_vectors_arpc3_positive.append(new_ch1_vector)
            ch2_vectors_arpc3_positive.append(new_ch2_vector)
#             print(new_ch2_vector)
            channel_0_2_separations_arpc3_positive.append(channel_0_2_separation_track)
            channel_0_1_separations_arpc3_positive.append(channel_0_1_separation_track)  
            channel_1_2_separations_arpc3_positive.append(channel_1_2_separation_track)  
            channel_0_2_separation_track = np.array(channel_0_2_separation_track)
            channel_0_1_separation_track = np.array(channel_0_1_separation_track)
            channel_1_2_separation_track = np.array(channel_1_2_separation_track)
            final_separations_ap2arpc3_arpc3_positive.append(channel_0_2_separation_track[~np.isnan(channel_0_2_separation_track)][-1])      
            initial_separations_ap2arpc3_arpc3_positive.append(channel_0_2_separation_track[~np.isnan(channel_0_2_separation_track)][0])   
            max_separation_ap2_arpc3_arpc3_positive.append(np.nanmax(channel_0_2_separation_track))
            max_separation_ap2_dnm2_arpc3_positive.append(np.nanmax(channel_0_1_separation_track))
            max_separation_dnm2_arpc3_arpc3_positive.append(np.nanmax(channel_1_2_separation_track))
            num_frames_associated_arpc3_positive.append(mode[1][0])
            pval_arpc3_ccps_arpc3_positive.append(pval_arpc3_detection)
            maximum_ap2_intensity_arpc3_positive.append(np.nanmax(new_ch0_vector))
            maximum_dnm2_intensity_arpc3_positive.append(np.nanmax(new_ch1_vector))
            maximum_arpc3_intensity_arpc3_positive.append(np.nanmax(new_ch2_vector))      
            time_ap2_appearance_to_dnm2_peak_arpc3_positive.append(dnm2_peak)
            time_arpc3_appearance_to_dnm2_peak_arpc3_positive.append(int(frames_in_track_first_channel[dnm2_peak]) - int(frames_in_track_second_channel[0]))
            experiment_number_arpc3_positive.append(experiment_number)
            
            dnm2_arpc3_peak_time_difference_arpc3_positive.append(np.nanargmax(ch1_amplitudes)-np.nanargmax(ch2_amplitudes))
            
            # concatenate all track positions and amplitudes
            track_x_positions = [new_ch0_x, new_ch1_x, new_ch2_x]
            track_y_positions = [new_ch0_y, new_ch1_y, new_ch2_y]
            channel_intensity_vectors = [new_ch0_vector, new_ch1_vector, new_ch2_vector]   
             
            ap2_x_positions = new_ch0_x
            ap2_y_positions = new_ch0_y     
            
            num_before=5
            ap2movement = []
            for dnm2_index in range(dnm2_peak-num_before,dnm2_peak):

                try: 
                    
                    if ap2_x_positions[dnm2_index] != 0 and ap2_x_positions[dnm2_index+1] != 0:
                        
                        ap2temp = np.sqrt((0.108*(ap2_x_positions[dnm2_index+1]-ap2_x_positions[dnm2_index]))**2 + (0.108*(ap2_y_positions[dnm2_index+1]-ap2_y_positions[dnm2_index]))**2)

                        ap2movement.append(ap2temp)
                        
                    else:
                        
                        ap2movement.append(np.NaN)

                except Exception:

                    ap2movement.append(np.NaN)

            average_ap2_movement_before_dnm2_peak_arpc3_positive.append(np.nanmean(ap2movement))            

            num_after=5
            ap2movement = []
            for dnm2_index in range(dnm2_peak,dnm2_peak+num_after):

                try: 
                    
                    if ap2_x_positions[dnm2_index] != 0 and ap2_x_positions[dnm2_index+1] != 0:
                        
                        ap2temp = np.sqrt((0.108*(ap2_x_positions[dnm2_index+1]-ap2_x_positions[dnm2_index]))**2 + (0.108*(ap2_y_positions[dnm2_index+1]-ap2_y_positions[dnm2_index]))**2)

                        ap2movement.append(ap2temp)
                        
                    else:
                        
                        ap2movement.append(np.NaN)

                except Exception:

                    ap2movement.append(np.NaN)

            average_ap2_movement_after_dnm2_peak_arpc3_positive.append(np.nanmean(ap2movement)) 
            
        elif no_neighbors==True: # if there is no mode then the event is arpc3 negative

            num_without_mode += 1 
            experiment_number_arpc3_negative.append(experiment_number)
            ccps_arpc3_negative.append(ap2dmn2_tracks_in_experiment[track_num])
            ap2_lifetime_arpc3_negative.append(len(ap2_int))

            ch0_vectors_arpc3_negative.append(ap2_int)
            ch1_vectors_arpc3_negative.append(dnm2_int)

            filtered_dnm2_signal = list(list(signal.sosfilt(sos, dnm2_int)) + [0, 0, 0, 0, 0])
    
            dnm2_peak = signal.find_peaks(filtered_dnm2_signal, 
                                          distance=best_fit_peak_params[0], 
                                          height=best_fit_peak_params[1],
                                          width=best_fit_peak_params[2])[0][0]
            
            time_ap2_appearance_to_dnm2_peak_arpc3_negative.append(dnm2_peak)
            
                        
            channel_0_1_separation_track = []
            
            for frame in range(len(ch0_x)):
                
                channel_0_1_separation_track.append(np.sqrt((0.108*(ch0_x[frame]-ch1_x[frame]))**2 + (0.108*(ch0_y[frame]-ch1_y[frame]))**2)) # ap2-dnm2 separation

            channel_0_1_separations_arpc3_negative.append(channel_0_1_separation_track)
         
            ap2_x_positions = ch0_x
            ap2_y_positions = ch0_y     
        
            num_before=5
            ap2movement = []
            for dnm2_index in range(dnm2_peak-num_before,dnm2_peak):

                try: 

                    ap2temp = np.sqrt((0.108*(ap2_x_positions[dnm2_index+1]-ap2_x_positions[dnm2_index]))**2 + (0.108*(ap2_y_positions[dnm2_index+1]-ap2_y_positions[dnm2_index]))**2)

                    ap2movement.append(ap2temp)

                except Exception:

                    ap2movement.append(np.NaN)

            average_ap2_movement_before_dnm2_peak_arpc3_negative.append(np.nanmean(ap2movement))            

            num_after=5
            ap2movement = []
            for dnm2_index in range(dnm2_peak,dnm2_peak+num_after):

                try: 

                    ap2temp = np.sqrt((0.108*(ap2_x_positions[dnm2_index+1]-ap2_x_positions[dnm2_index]))**2 + (0.108*(ap2_y_positions[dnm2_index+1]-ap2_y_positions[dnm2_index]))**2)

                    ap2movement.append(ap2temp)

                except Exception:

                    ap2movement.append(np.NaN)

            average_ap2_movement_after_dnm2_peak_arpc3_negative.append(np.nanmean(ap2movement))             
          
    print('fraction of events with arpc3', str(num_with_mode/(num_with_mode+num_no_neighbors)))
    fraction_arpc3_positive.append(num_with_mode/(num_with_mode+num_no_neighbors))   

current experiment number: 22
number of ap2 tracks: 212
number of arpc3 tracks: 13557
iterating through tracks in experiment


Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice


fraction of events with arpc3 0.7393617021276596
current experiment number: 24
number of ap2 tracks: 210
number of arpc3 tracks: 12428
iterating through tracks in experiment


Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice


fraction of events with arpc3 0.7608695652173914
current experiment number: 26
number of ap2 tracks: 197
number of arpc3 tracks: 14836
iterating through tracks in experiment


...

fraction of events with arpc3 0.7697368421052632
current experiment number: 68
number of ap2 tracks: 208
number of arpc3 tracks: 13944
iterating through tracks in experiment


Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice


fraction of events with arpc3 0.8021390374331551
current experiment number: 70
number of ap2 tracks: 154
number of arpc3 tracks: 13399
iterating through tracks in experiment


Mean of empty slice


fraction of events with arpc3 0.7591240875912408


# save all results

In [60]:
fraction_arpc3_positive

[0.7393617021276596,
 0.7608695652173914,
 0.7094972067039106,
 0.6834170854271356,
 0.7955555555555556,
 0.6629213483146067,
 0.8,
 0.7697368421052632,
 0.8021390374331551,
 0.7591240875912408]

In [61]:
arrays_to_save = ['ch0_vectors_arpc3_positive',
                  'ch1_vectors_arpc3_positive',
                  'ch2_vectors_arpc3_positive',
                  'channel_0_1_separations_arpc3_positive',
                  'channel_0_2_separations_arpc3_positive',
                  'channel_1_2_separations_arpc3_positive',
                  'final_separations_ap2arpc3_arpc3_positive',
                  'initial_separations_ap2arpc3_arpc3_positive',
                  'ap2_arpc3_separation_at_dnm2_peak_arpc3_positive',
                  'ap2_arpc3_separation_at_arpc3_peak_arpc3_positive',
                  'ap2_arpc3_separation_at_ap2_peak_arpc3_positive',
                  'ap2_lifetime_arpc3_positive',
                  'arpc3_lifetime_arpc3_positive',
                  'max_separation_ap2_arpc3_arpc3_positive',
                  'max_separation_ap2_dnm2_arpc3_positive',
                  'max_separation_dnm2_arpc3_arpc3_positive',
                  'num_frames_associated_arpc3_positive',
                  'time_ap2_appearance_to_dnm2_peak_arpc3_positive',
                  'maximum_ap2_intensity_arpc3_positive',
                  'maximum_dnm2_intensity_arpc3_positive',
                  'maximum_arpc3_intensity_arpc3_positive',
                  'pval_arpc3_ccps_arpc3_positive',
                  'time_arpc3_appearance_to_dnm2_peak_arpc3_positive',
                  'fraction_arpc3_positive',
                  'ccps_arpc3_positive',
                  'experiment_number_arpc3_positive',
                  'average_ap2_movement_before_dnm2_peak_arpc3_positive',
                  'average_ap2_movement_after_dnm2_peak_arpc3_positive',
                  'average_ap2_movement_before_dnm2_peak_arpc3_negative',
                  'average_ap2_movement_after_dnm2_peak_arpc3_negative',
                  'ap2_arpc3_separation_average_arpc3_positive',
                  'ch0_vectors_arpc3_negative',
                  'ch1_vectors_arpc3_negative',
                  'channel_0_1_separations_arpc3_negative',
                  'ccps_arpc3_negative',
                  'time_ap2_appearance_to_dnm2_peak_arpc3_negative',
                  'ap2_lifetime_arpc3_negative',
                  'experiment_number_arpc3_negative',
                  'significant_arpc3_at_dnm2_peak_arpc3_positive']

merged_results = {}
for array in arrays_to_save:
    
    merged_results[array] = locals()[array]   

In [62]:
len(ccps_arpc3_positive), len(ccps_arpc3_negative)

(1298, 441)

In [63]:
len(num_frames_associated_arpc3_positive), len(ap2_arpc3_separation_at_dnm2_peak_arpc3_positive), len(significant_arpc3_at_dnm2_peak_arpc3_positive)

(1298, 1298, 1298)

In [64]:
with open(unique_user_saved_outputs+'/dataframes/merged_ap2arpc3_shock_merged_before_data_zeropadding', 'wb') as f:
    pickle.dump(merged_results, f)   
    
np.save(unique_user_saved_outputs+'/dataframes/ccps_arpc3_shock_merge_before_negative_zeropadding', ccps_arpc3_negative)
    
np.save(unique_user_saved_outputs+'/dataframes/ccps_arpc3_shock_merge_before_positive_zeropadding', ccps_arpc3_positive)